In [2]:
import pandas as pd
import json
import webbrowser
import time
import requests

decode_serviceKey = 'NwS7t35ZvejHIExKbCymXgmyoLk0WLkZKDslnArD0xyBdUm3ohgWtefXVg25j9L9z3UdyDXXWbfIVjwGfOFArw=='    
ednode_serviceKey = 'NwS7t35ZvejHIExKbCymXgmyoLk0WLkZKDslnArD0xyBdUm3ohgWtefXVg25j9L9z3UdyDXXWbfIVjwGfOFArw%3D%3D'

### getCommercialArea(행정동코드)
def getCommercialArea(signguCd, adongNm):
    ### getDivId(구코드, 찾을 행정동)
    def getDivId():
        #signguCd = '11590' 동작구
        webbrowser.open("http://apis.data.go.kr/B553077/api/open/sdsc2/baroApi")
        time.sleep(1.5) #임시 비동기 처리
            
        url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/baroApi?resId=dong&catId=admi'+'&serviceKey=' + ednode_serviceKey + '&signguCd=' + signguCd + '&type=json'
            
        response_json = requests.get(url, allow_redirects=False)
        response = json.loads(response_json.content)
        responseArr = response.get('body').get('items')
            
        _return = ''
        for list in responseArr:
            if str(list.get('adongNm')) == adongNm :
                _return = list.get('adongCd')
                
            if(_return == ''):
                _return = 'Not found'
            
        return _return
        
    webbrowser.open("http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong")
    time.sleep(2) #임시 비동기 처리
        
    #params
    url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong'
    divId='adongCd'
    key = getDivId()
    params ={'serviceKey' : decode_serviceKey, 'pageNo' : '1', 'numOfRows' : '1000', 'divId' : divId, 'key' : key, 'type' : 'json' }
        
    response_json = requests.get(url, params=params, allow_redirects=False)
    response = json.loads(response_json.content)
    responseArr = response.get('body').get('items')

    _return = pd.DataFrame(responseArr)
        
    #테스트 필요
    page = response.get('body').get('totalCount') // 1000 + 1
    if page > 1 :
        for p in range(1, page):
            params['pageNo'] = str(p)
            add_response_json = requests.get(url, params=params, allow_redirects=False)
            add_response = json.loads(add_response_json.content)
            add_responseArr = add_response.get('body').get('items')
            responseArr = responseArr + add_responseArr
            
    return _return
    
### 필요한 인자가 시군구코드, 찾을행정동
class CommercialArea:
    def __init__(self, signguCd, adongNm):
        self._signguCd = signguCd
        self._adongNm = adongNm
        self._data = getCommercialArea(self._signguCd, self._adongNm)
        
    def getDf(self):
        return self._data
##이런식으로 사용하면 됨
#test = CommercialArea('11590', '상도1동')
#test.getDf()
##리펙토링 마렵네요..

In [3]:
# -*- coding: utf-8 -*-
"""



"""

import pandas as pd

def load_data(univ):
    excel_data = pd.read_csv("대학리스트.csv")
    result_data= excel_data[excel_data['대학']==univ]
    return str(result_data['시군구코드'].values[0]), result_data['행정동이름'].values[0]

    
    
def main():
    a, b = load_data('숭실대학교')
    print(type(str(a)))
    print(type(b))


    
if __name__ =='__main__':
    main()



class LoadUnivData:
    def __init__(self, univ):
        self.univ = univ
        self.gucode, self.dongname = load_data(self.univ)
        
        
    def loadUniv(self):
        return self.gucode, self.dongname
        
        

<class 'str'>
<class 'str'>


In [26]:
from jungeol import LoadUnivData
from jongGang import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
soongsil = LoadUnivData('숭실대학교') #LoadUnivData 객체 생성
soongsil_gucode, soongsil_dongname = soongsil.loadUniv()

soongsil_CA = CommercialArea(soongsil_gucode, soongsil_dongname) #CommercialArea 객체 생성
soongsil_CA.getDf()

### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고

print(soongsil_CA.getDf().loc[0,:])


bizesId                        11756204
bizesNm                           순에스테틱
brchNm                                 
indsLclsCd                            F
indsLclsNm                        생활서비스
indsMclsCd                          F01
indsMclsNm                      이/미용/건강
indsSclsCd                       F01A03
indsSclsNm                      비만/피부관리
ksicCd                                 
ksicNm                                 
ctprvnCd                             11
ctprvnNm                          서울특별시
signguCd                          11590
signguNm                            동작구
adongCd                      1159053000
adongNm                            상도1동
ldongCd                      1159010200
ldongNm                             상도동
lnoCd               1159010200104960022
plotSctCd                             1
plotSctNm                            대지
lnoMnno                             496
lnoSlno                              22
lnoAdr             서울특별시 동작구 상도동 496-22


In [24]:
soongsil=soongsil_CA.getDf()[["bizesNm","indsLclsNm","indsSclsNm","rdnmAdr","lon","lat"]]
soongsil.to_csv("soongsil.csv",mode="w",encoding='euc-kr')

In [1]:
!pip install folium

In [42]:
import folium
import pandas as pd

In [43]:
map_osm = folium.Map(location = [37.49411614, 126.9543037], zoom_start = 16)
map_osm.save('09_gate_map.html')

In [44]:
soongsil_map = pd.read_csv('soongsil.csv', encoding = 'cp949', engine = 'python')

In [45]:
map_CB = folium.Map(location = [37.49411614, 126.9543037], zoom_start = 15)

In [46]:
for i, bizesNm in soongsil_map.iterrows():
    folium.Marker(location = [bizesNm['lat'], bizesNm['lon']], popup = bizesNm['bizesNm'], icon = folium.Icon(color = 'red', icon = 'star')).add_to(map_CB)

In [47]:
map_CB.save('map_CB.html')

In [48]:
import webbrowser
webbrowser.open('map_CB.html')

True